## Read training data

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd

train_mouth = True
train_smile = True
train_brows = True



# Params:
#   - datafr:      Dataframe to add to.
#   - fname:       Name of CSV file
#   - target:      Target value
#   - add_target:  If true, target value from CSV is overwritten
def read_and_add(datafr, fname, target, add_target):
    df_temp = pd.read_csv("training_data/" + fname + ".csv")
    if (add_target == True):
        df_temp['target'] = target 
    df_temp = df_temp.dropna(how='any')
    return datafr.append(df_temp, ignore_index=True)

# Read mouth open/close training data.
df_mouth = pd.DataFrame();
df_mouth = read_and_add(df_mouth, "mouth_0.0", 0.0, True)
df_mouth = read_and_add(df_mouth, "mouth_0.2", 0.2, True)
df_mouth = read_and_add(df_mouth, "mouth_0.4", 0.4, True)
df_mouth = read_and_add(df_mouth, "mouth_0.6", 0.6, True)
df_mouth = read_and_add(df_mouth, "mouth_0.8", 0.8, True)
df_mouth = read_and_add(df_mouth, "mouth_1.0", 1.0, True)

# Read smile/frown training data.
df_smile = pd.DataFrame();
df_smile = read_and_add(df_smile, "smile_0.0", 0.0, False)
df_smile = read_and_add(df_smile, "smile_1.0", 1.0, False)

# Read brows up/down training data.
df_brows = pd.DataFrame();
df_brows = read_and_add(df_brows, "brows_0.0", 0.0, False)
df_brows = read_and_add(df_brows, "brows_0.5", 0.5, False)
df_brows = read_and_add(df_brows, "brows_1.0", 1.0, False)

df_mouth.tail(10)

## Summary of data

In [ ]:
print(df_mouth.describe())

## Train/Test split

In [ ]:
import numpy
import sklearn
from sklearn.model_selection import train_test_split

features_mouth = df_mouth.iloc[:, : 468]
features_smile = df_smile.iloc[:, : 468]
features_brows = df_brows.iloc[:, : 468]

targets_mouth = df_mouth['target'].values 
targets_smile = df_smile['target'].values 
targets_brows = df_brows['target'].values 

x_train_mouth, x_test_mouth, y_train_mouth, y_test_mouth = train_test_split(features_mouth, targets_mouth, train_size=0.8, random_state=3)
x_train_smile, x_test_smile, y_train_smile, y_test_smile = train_test_split(features_smile, targets_smile, train_size=0.8, random_state=3)
x_train_brows, x_test_brows, y_train_brows, y_test_brows = train_test_split(features_brows, targets_brows, train_size=0.8, random_state=3)

print (targets_mouth)
print (targets_smile)
print (targets_brows)

## Model setup

In [ ]:
import tensorflow as tf
import keras

from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import optimizers

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

Setting the model's layers. 468 facial features as returned by ARCore.


In [ ]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(468,)))
model.add(Dropout(0.05))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.05))
model.add(Dense(1, activation='linear'))

model_mouth = model
model_smile = model
model_brows = model

In [ ]:
# Print model summary.
model.summary()

# Draw model graph.
# from keras.utils import plot_model
# plot_model(model, to_file='model.png', show_shapes=True)

In [ ]:
model.compile(loss='mse',
              optimizer='Adam',
              metrics=['mse','mae'])

## Model training

In [ ]:
import time

# 256 is a good value
train_epochs = 256
# 128 is a good value
train_batch_size = 128

if train_mouth:
    start = time.time()
    history_mouth = model_mouth.fit(x_train_mouth, y_train_mouth,
                        batch_size=train_batch_size,
                        epochs=train_epochs,
                        validation_data=(x_test_mouth, y_test_mouth))
    model_mouth.evaluate(x_test_mouth, y_test_mouth, verbose=1)
    model_mouth.save('model_mouth.h5')
    end = time.time()
    print("Training done, took " + "{0:.4f}".format(end - start) + " seconds.\n\n")
    
if train_smile:
    start = time.time()
    history_smile = model_smile.fit(x_train_smile, y_train_smile,
                        batch_size=train_batch_size,
                        epochs=train_epochs,
                        validation_data=(x_test_smile, y_test_smile))
    model_smile.evaluate(x_test_smile, y_test_smile, verbose=1)
    model_smile.save('model_smile.h5')
    end = time.time()
    print("Training done, took " + "{0:.4f}".format(end - start) + " seconds.\n\n")
    
if train_brows:
    start = time.time()
    history_brows = model_brows.fit(x_train_brows, y_train_brows,
                        batch_size=train_batch_size,
                        epochs=train_epochs,
                        validation_data=(x_test_brows, y_test_brows))
    model_brows.evaluate(x_test_brows, y_test_brows, verbose=1)
    model_brows.save('model_brows.h5')
    end = time.time()
    print("Training done, took " + "{0:.4f}".format(end - start) + " seconds.\n\n")


In [ ]:
import matplotlib.pyplot as plt

# history_brows
# history_smile
# history_mouth

plt.plot(history_brows.history['mean_absolute_error'])
plt.plot(history_brows.history['val_mean_absolute_error'])

plt.title('model accuracy')
plt.ylabel('MAE/Loss')
plt.xlabel('epoch')
plt.legend(['train_mae', 'test_mae'], loc='upper left')
plt.rcParams["figure.figsize"] = [16,9]
plt.savefig('smile_mae.png')
plt.show()

## Try some predictions

In [ ]:
# np.set_printoptions(suppress=True)
# print(model.predict(smile.iloc[:, : 468])[0:999])
# np.set_printoptions(suppress=False)